<a href="https://colab.research.google.com/github/NicolasCunha328/Imers-o-IA-Alura/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip -q install google-genai

In [7]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [8]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [11]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais e o site da Alura:** A Alura costuma anunciar novas edições de seus cursos e imersões em suas redes sociais (como LinkedIn, Instagram, Twitter) e no site oficial.
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter, você receberá informações sobre os próximos eventos, cursos e imersões.
*   **Verificar a página de Imersões da Alura:** Visite a página de Imersões da Alura no site para verificar se há alguma edição futura da Imersão IA com Google Gemini já programada.

In [12]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [13]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [14]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [15]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [16]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [17]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [18]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [19]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [20]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [21]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [22]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: agentes de IA
Maravilha! Vamos então criar o post sobre novidades em agentes de IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer uma visão geral dos lançamentos mais relevantes sobre agentes de IA no último mês, farei algumas pesquisas no Google para identificar os tópicos mais comentados e promissores.
> 
> 
> Com base nas pesquisas, aqui estão alguns lançamentos e desenvolvimentos relevantes em agentes de IA nos últimos meses:
> 
> 1.  **Adoção generalizada de agentes de IA em vários setores:** Em 2025, os agentes de IA estão se tornando protagonistas na transformação digital, com setores como e-commerce, atendimento ao cliente, marketing e finanças incorporando esses agentes para automatizar tarefas, analisar dados e melhorar a eficiência operacional. [1]
> 2.  **Agentes de IA remodelando as indústrias:** Agentes de IA estão combinando maior poder de síntese de dados e capacidades aprimoradas de tomada de decisão, execução de tarefas e interação com múltiplas ferramentas com o mínimo de intervenção humana. Eles estão transformando setores como governo, saúde e finanças, melhorando a operação, aprimorando a experiência e reduzindo os custos. [2]
> 3.  **Integração e orquestração de agentes de IA:** Empresas como IBM, Anthropic, Amazon, Google e OpenAI estão anunciando atualizações e lançamentos que prometem revolucionar o cenário da inteligência artificial, influenciando tanto o setor corporativo quanto a realidade brasileira com ferramentas inovadoras e integração avançada entre agentes. A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções inovadoras para integrar, customizar e implantar agentes, além de disponibilizar agentes pré-construídos para áreas como RH, vendas e compras. [3]
> 4.  **Avanços e tendências em agentes de IA:** Os agentes de IA estão se tornando mais avançados, fazendo a ponte para a Inteligência Artificial Geral (AGI). Ao contrário dos sistemas de IA tradicionais projetados para tarefas limitadas, esses agentes podem aprender continuamente, adaptar-se a ambientes dinâmicos e tomar decisões independentes com base em dados em tempo real. [11]
> 5.  **Oportunidades para desenvolvedores com agentes de IA:** Desenvolvedores terão a chance de criar APIs específicas para agentes de IA, interfaces personalizadas que apresentam conteúdo adaptado às preferências individuais dos usuários e sistemas de microtransações que permitam interações comerciais entre agentes de IA. [8]
> 
> Esses lançamentos e desenvolvimentos refletem um crescente interesse e investimento em agentes de IA, com o potencial de transformar uma ampla gama de setores e aplicações.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com certeza! Com base nos lançamentos e desenvolvimentos recentes em agentes de IA, preparei um plano detalhado para um post de mídia social, focando no tema mais relevante e promissor.
> 
> **Pesquisas Adicionais:**
> 
> Para complementar as informações fornecidas, realizei algumas pesquisas adicionais para aprofundar meu conhecimento sobre o tema e identificar os pontos mais relevantes para o público:
> 
> 
> **Tema Escolhido:** Integração e Orquestração de Agentes de IA
> 
> **Por que este tema?**
> 
> *   **Relevância:** A capacidade de integrar e orquestrar diferentes agentes de IA é fundamental para desbloquear todo o potencial dessa tecnologia. As empresas precisam de soluções que permitam combinar agentes pré-construídos com agentes personalizados para atender às suas necessidades específicas.
> *   **Novidade:** Os lançamentos recentes da IBM (watsonx Orchestrate) e outras empresas indicam um foco crescente nessa área, com o objetivo de facilitar a adoção e o gerenciamento de agentes de IA.
> *   **Impacto:** A integração e orquestração de agentes de IA têm o potencial de transformar diversos setores, automatizando processos complexos, melhorando a tomada de decisões e impulsionando a inovação.
> 
> **Pontos Relevantes:**
> 
> *   **O que são agentes de IA e como eles funcionam?** Explicar o conceito de agentes de IA e como eles se diferenciam dos sistemas de IA tradicionais.
> *   **Benefícios da integração e orquestração de agentes de IA:** Eficiência, automação, inovação, personalização, tomada de decisões aprimorada.
> *   **Desafios da implementação de agentes de IA:** Ética, privacidade, segurança, viés algorítmico, complexidade técnica, altos custos computacionais.
> *   **Exemplos de uso:** RH, vendas, compras, atendimento ao cliente, análise financeira, descoberta de medicamentos, etc.
> *   **Oportunidades para desenvolvedores:** Criação de APIs específicas para agentes de IA, interfaces personalizadas, sistemas de microtransações.
> *   **Tendências futuras:** Agentes de IA mais avançados, integração com IoT e dispositivos inteligentes, avanços na aprendizagem de agentes de IA.
> *   **Impacto no mercado de trabalho:** Necessidade de requalificação profissional, surgimento de novas funções (gestores de equipes de agentes de IA).
> 
> **Plano para o Post:**
> 
> 1.  **Título:** "Agentes de IA: A Revolução da Automação Inteligente Está Apenas Começando"
> 2.  **Introdução:**
>     *   Contextualizar o crescimento da adoção de agentes de IA em diversos setores.
>     *   Apresentar a integração e orquestração de agentes de IA como um passo crucial para a transformação digital.
> 3.  **O que são Agentes de IA:**
>     *   Definir agentes de IA e explicar como eles funcionam (percepção, raciocínio, ação).
>     *   Diferenciar agentes de IA de sistemas de IA tradicionais (foco em autonomia e adaptação).
> 4.  **Benefícios da Integração e Orquestração:**
>     *   Automatização de tarefas complexas e repetitivas.
>     *   Melhoria da tomada de decisões com base em dados em tempo real.
>     *   Aumento da eficiência operacional e redução de custos.
>     *   Personalização de experiências para clientes e funcionários.
>     *   Impulsionamento da inovação e criação de novos produtos e serviços.
> 5.  **Desafios da Implementação:**
>     *   Questões éticas e de privacidade relacionadas ao uso de agentes de IA.
>     *   Riscos de viés algorítmico e discriminação.
>     *   Complexidade técnica e necessidade de profissionais qualificados.
>     *   Custos de implementação e manutenção.
> 6.  **Exemplos de Uso:**
>     *   Apresentar exemplos práticos de como os agentes de IA estão sendo usados em diferentes setores (e-commerce, atendimento ao cliente, finanças, saúde, etc.).
>     *   Mencionar casos de sucesso de empresas que já implementaram agentes de IA com sucesso.
> 7.  **Oportunidades para Desenvolvedores:**
>     *   Destacar as oportunidades para desenvolvedores na criação de APIs, interfaces e sistemas de microtransações para agentes de IA.
>     *   Incentivar a comunidade de desenvolvedores a explorar esse campo em expansão.
> 8.  **Tendências Futuras:**
>     *   Discutir as tendências futuras em agentes de IA, como agentes mais avançados, integração com IoT e dispositivos inteligentes, e avanços na aprendizagem de agentes de IA.
>     *   Explorar o impacto dos agentes de IA no mercado de trabalho e na sociedade como um todo.
> 9.  **Conclusão:**
>     *   Reafirmar o potencial transformador dos agentes de IA.
>     *   Incentivar as empresas a explorar e adotar agentes de IA de forma estratégica e responsável.
>     *   Abrir espaço para comentários e perguntas dos seguidores.
> 
> Espero que este plano seja útil! Avise-me se precisar de mais alguma coisa.
> 


--------------------------------------------------------------


Exception in thread Thread-13 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 3 (Redator) ---



ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-22' coro=<AsyncClient.aclose() done, defined at /usr/local/lib/python3.11/dist-packages/httpx/_client.py:1978> exception=RuntimeError('Event loop is closed')>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection_pool.py", line 345, in _close_connections
    await connection.aclose()
  File "/usr/local/lib/python3.11/dist-packages/httpcore/_async/connection.py", line 173, in aclose
    await self._connection.aclose()
  File "/usr/local/lib/pytho

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



--------------------------------------------------------------
